In [ ]:
import os
import pickle

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Load Data

In [ ]:
# Load DataFrame Generated by StaticWindowResults.ipynb
directory = "INSERT DIRECTORY"

with open(os.path.join(directory, 'INSERT FILENAME'), 'rb') as f:
    results_df = pickle.load(f)
results_df.tail()

# Preprocess Data

In [ ]:
# Generate Score Averages
results_df_mean = results_df.reset_index()
results_df_mean = results_df_mean.groupby(['SNR', 'Thresh']).agg('mean')
results_df_mean.head()

In [ ]:
# Generate Score Deviations
results_df_std = results_df.reset_index()
results_df_std = results_df_std.groupby(['SNR', 'Thresh']).agg('std')
results_df_std.head()

In [ ]:
# Calculate AUC score for ROC curves
from sklearn.metrics import auc

def auc_calc(group):
    group = group.sort_values(by='FPR')
    
    fprs = group.FPR.values
    tprs = group.TPR.values
    val = auc(fprs, tprs)
    
    return val
    

auc_df = results_df.groupby(['SNR', 'trace_idx']).apply(auc_calc).rename('AUC')
auc_df = auc_df.to_frame()
auc_df

## Figures

In [ ]:
temp_results_df = results_df.reset_index()
columns = list(temp_results_df.columns)

for snr in temp_results_df.SNR.unique():
    for idx in temp_results_df['trace_idx'].unique():
        values1 = [snr, [-10], idx, [1], [1], [0], [0]]
        values2 = [snr, [10], idx, [0], [0], [1], [1]]
        temp_results_df = pd.concat([temp_results_df, pd.DataFrame(dict(zip(columns, values1)))])
        temp_results_df = pd.concat([temp_results_df, pd.DataFrame(dict(zip(columns, values2)))])
        
temp_results_df.tail() 

In [ ]:
roc_mean_df = temp_results_df.groupby(['SNR', 'Thresh']).agg('mean').reset_index()

In [ ]:
# Plot of ROC curves and their corresponding AUC scores
fig, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=600)
sns.lineplot(data=roc_mean_df, x='FPR', y='TPR', hue=roc_mean_df.SNR.astype('string'), palette='tab10', ax=axs[0])

sns.barplot(data=auc_df.reset_index(), x='SNR', y='AUC', palette='tab10', ax=axs[1])

axs[0].set_xlabel('FPR', weight='bold')
axs[1].set_xlabel('SNR', weight='bold')
axs[0].set_ylabel('TPR', weight='bold')
axs[1].set_ylabel('AUC', weight='bold')

axs[0].tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    axs[0].spines[axis].set_linewidth(1.5)
axs[1].tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    axs[1].spines[axis].set_linewidth(1.5)
    
axs[0].annotate('(a)', xy=(0, 0), xytext=(-0.3, 1.1), weight='bold')
axs[1].annotate('(b)', xy=(0.1, 0), xytext=(-2.1, 1.095), weight='bold')

fig.tight_layout()

# fig.savefig('MA ROC AUC.png')

In [ ]:
auc_df.groupby('SNR').agg('mean')